<a href="https://colab.research.google.com/github/MattoDemente/CryptoTrading/blob/master/Indicators_Tagging_Binance_BTC_TUSD_1H_24062020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install backtesting

from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
from backtesting.lib import resample_apply


In [4]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import time
import os
import re
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
from termcolor import colored, cprint
from scipy.signal import argrelextrema
from datetime import timedelta
#%matplotlib inline
#%matplotlib notebook
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm


from google.colab import drive
drive.mount('/content/gdrive')


TypeError: ignored

Data Source https://www.CryptoDataDownload.com


In [ ]:
def load_crncy(path, kind='train'):
    """Load Bitcoin MXN trade info"""
    
    data_path = os.path.join(path, 'Binance_BTCUSDT_1h_1445212800000-1592524800000.csv')
    df = pd.read_csv(data_path, sep = ",",skiprows = 0)
    #df = pq.read_pandas(data_path, columns=['open_time',
    #                                        'open',
    #                                        'high',
    #                                        'low',
    #                                        'close',
    #                                        'volume',
    #                                        'quote_asset_volume',
    #                                        'number_of_trades',
    #                                        'taker_buy_asset_volume',
    #                                        'taker_buy_quote_asset_volume'
    #                                        ]).to_pandas()

    print('First rows of features')
    
    print(df.head)

    return df


In [ ]:
df = load_crncy('/content/gdrive/My Drive/Colab Notebooks/BINANCE/data/')    

In [ ]:
df.dtypes

In [ ]:
df['date_time'] = df.Timestamp.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df.index = df.date_time

In [ ]:
df['date_time'].dtype

In [ ]:
df.dtypes

In [ ]:
df.index

# Aquí va el código para arreglar huecos en el índice

In [ ]:
ts = pd.date_range(df.index.min(), df.index.max(), freq='1H')

In [ ]:
ts

In [ ]:
len(ts)

In [ ]:
df[df.index.duplicated()]

In [ ]:
df = df[~df.index.duplicated()]

In [ ]:
df.loc['2017-08-17 15:00:00']

In [ ]:
df = df.reindex(ts).fillna(method='ffill').rename_axis('open_time').reset_index()

In [ ]:
df.index = df.open_time

In [ ]:
#df['open_time'] = df.open_time.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %I-%p'))
#df.index = df.DateTime
#df = df.sort_index(ascending=True)
df.head()

In [ ]:
df = df.drop(['open_time','Timestamp','date_time'], axis=1)

In [ ]:
df.loc['2017-09-06 15:00:00']

Hacer correlación de los dos volumenes BTC y MXN (verificar si es 1)
Agregar features de Volumne pro plataforma 

In [ ]:
data_frame = df.copy(deep=True)

In [ ]:
firstVI = data_frame.first_valid_index()
lastVI = data_frame.last_valid_index()

In [ ]:
#symbol='BTCMXN'
# rename the column header with symbol name
data_frame = data_frame.rename(columns={'open_time':'DateTime',
                                        'open':'Open',
                                        'high':'High',
                                        'low':'Low',
                                        'close':'Close'
                                        })
data_frame.dropna(inplace=True)
data_frame = data_frame.sort_index(ascending=True) #SORT 2019/10/02

In [ ]:
len(data_frame)

In [ ]:
##Find missing values
data_frame[pd.isnull(data_frame['Open'])]

In [ ]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 12}
plt.rc('font',**font)        

data_frame.plot(figsize=(15,10),subplots=True)

* Cleanup unnecessary columns

In [ ]:
#data_frame = data_frame.rename(columns={'DateTime':'Date'})
data_frame = data_frame.drop(['Volume','Quote Asset Volume', 'Number of trades', 'Take buy Asset volume' ], axis=1)

# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

In [ ]:
data_frame = data_frame.resample('1H').mean()

In [ ]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 12}
plt.rc('font',**font)        

data_frame.plot(figsize=(15,10),subplots=True)

* Calculation of Commodity Channel Index

In [ ]:
len(data_frame)

In [ ]:
start_DT = data_frame.first_valid_index()
end_DT = data_frame.last_valid_index()

In [ ]:
firstVI = data_frame.first_valid_index()
lastVI = data_frame.last_valid_index()
for i in data_frame.index:
  if (i == firstVI) or (i == lastVI):
    print(f"{i}")

In [ ]:
d = timedelta(hours=1)
#d = timedelta(minutes=5)

i = firstVI
j = i - d
print(j)

In [ ]:
def plotDataset(data):

    data.set_index('Date',inplace=True)
    plt.plot(data['Close'], fillstyle='none',label='Close')  
    plt.plot(data[data['position'] == -1]['Close'],'v', color='red', fillstyle='full',label='Sell Signal Minor')
    plt.plot(data[data['position'] == -2]['Close'],'v', color='blue', fillstyle='full',label='Sell Signal Mayor')
    plt.plot(data[data['position'] == 0]['Close'],'o', color='black', fillstyle='full',label='No Action')
    plt.plot(data[data['position'] == 1]['Close'],'^', color='green', fillstyle='full',label='Buy Signal Minor')
    plt.plot(data[data['position'] == 2]['Close'],'^', color='yellow', fillstyle='full',label='Buy Signal Mayor')

    plt.legend(loc='upper left')
    plt.xticks(rotation='vertical')
    plt.show()

"""
def merge(data1,data2):
    df = data1.copy()
    for i in df.index:
        value = data2.loc[i,'position']
        if value != 0:
            df.loc[i,'position'] = value
    return df
"""

def merge(data1,data2):
    dfp1 = data1['position']
    dfp2 = data2['position']
    dfp = dfp2.combine(dfp1, (lambda x1, x2: x1 if x1 != 0 else x2), fill_value=0)
    return dfp


def convertMax(df,i,threshold,signal, comission):
    
    df_size = df['Close'].size
    actualValue = df.loc[i,'Close']

    #Señal de venta
    if (signal > 0):
      #derecha
      j = i + d
      while(j <= lastVI and abs(df.loc[j,'Close'] - actualValue) <= threshold and abs(df.loc[j,'Close'] - actualValue) > comission):    
        df.loc[j,'position'] = signal
        j += d
      #izquierda
      k = i - d
      while(k >= firstVI and abs(df.loc[k,'Close'] - actualValue) <= threshold and abs(actualValue - df.loc[k,'Close']) > comission):
        df.loc[k,'position'] = signal
        k -= d
      i = j- d    

    #Señal de compra
    if (signal < 0 ):
      #derecha
      j = i + d
      while(j <= lastVI and abs(df.loc[j,'Close'] - actualValue) <= threshold and abs(df.loc[j,'Close'] - actualValue) > comission):
        df.loc[j,'position'] = signal
        j += d
      #izquierda
      k = i - d
      while(k >= firstVI and abs(df.loc[k,'Close'] - actualValue) <= threshold and abs(actualValue - df.loc[k,'Close']) > comission):    
        df.loc[k,'position'] = signal
        k -= d
      i = j- d  

    return df,j

#APC

"""
def convertMin(df,i,threshold,signal):
    
    df_size = df['Close'].size
    actualValue = df.loc[i,'Close']
    #derecha
    j = i + d
    while(j <= lastVI and abs(df.loc[j,'Close'] - actualValue) <= threshold and (df.loc[j,'position'] == 0)):#!= 2 and df.at[j,'position'] != -2)):
        df.loc[j,'position'] = signal
        j += d
    #izquierda
    k = i - d
    while(k >= firstVI and abs(df.loc[k,'Close'] - actualValue) <= threshold and (df.at[k,'position'] == 0)):#!= 2 and df.at[k,'position'] != -2)):
        df.at[k,'position'] = signal
        k -= d
    i = j-d    
    return df,j
"""

def etiq(df, threshold=.15, comission=.42):
    print(f"Etiquetando:")
    #df_size = df['Close'].size
    i = firstVI
    while i < lastVI:
        position = data.loc[i,'position']
        if position == 2 or position == -2:
            closeValue = df.loc[i,'Close']
            #print(f"Percentage Close Value {closeValue *.02}")            
            #df,i = convertMax(df,i,(closeValue *.0225),signal)
            #df,i = convertMax(df,i,(closeValue *.1253),signal)
            df,i = convertMax(df,i,(closeValue * threshold),position, comission) #.399 is the highest delta for Nov 2017 through Jan 2018 highest variance
            #df,i = convertMax(df,i,0.5,signal)
        i += d
    i = firstVI
    
    while i < lastVI:
        position = data.at[i,'position']
        if position == 1 or position == -1:
            closeValue = df.loc[i,'Close']
            #print(f"Percentage Close Value {closeValue *.02}")
            #df,i = convertMax(df,i,(closeValue *.0225),signal)
            #df,i = convertMax(df,i,(closeValue *.1253),signal)
            df,i = convertMax(df,i,(closeValue * threshold),position, comission)
            #df,i = convertMin(df,i,0.25,signal)
        i += d
    
    return df

## TODO:
* Hacer split de 4 valores para el argrelextrema y revisar el umbral 
* Graficar curva con colores.

In [ ]:
def tagWParms(df , Lwindow=8760, Swindow=720): 


    df1 = df.copy(deep=True)

    dataMinor = df1.copy()
    dataMax = df1.copy()    
    #procesando ventas menores
    
    n = Swindow #un mes   
    
    dataMinor['mn'] = dataMinor.iloc[argrelextrema(dataMinor.Close.values, np.less_equal, order=Swindow)[0]]['Close']
    dataMinor['mx'] = dataMinor.iloc[argrelextrema(dataMinor.Close.values, np.greater_equal, order=Swindow)[0]]['Close']
    dataMinor['position'] = 0
    dataMinor.loc[dataMinor['mn'].notnull(),'position'] = 1.0 
    dataMinor.loc[dataMinor['mx'].notnull(),'position'] = -1.0 
    
    #dataMinor['mn'] = dataMinor.apply(lambda row: argrelextrema(dataMinor.Close.values, np.less_equal, order=Swindow))

    
    n = Lwindow #un año
    
    dataMax['mn'] = dataMax.iloc[argrelextrema(dataMax.Close.values, np.less_equal, order=Lwindow)[0]]['Close']    
    dataMax['mx'] = dataMax.iloc[argrelextrema(dataMax.Close.values, np.greater_equal, order=Lwindow)[0]]['Close']
    dataMax['position'] = 0
    dataMax.loc[dataMax['mn'].notnull(),'position'] = 2.0 
    dataMax.loc[dataMax['mx'].notnull(),'position'] = -2.0 
    
    
    #data = merge(dataMinor,dataMax)
    pos = merge(dataMinor,dataMax)
    df1['position'] = pos
    data = df1.copy(deep=True)
    print(f"VALORES ANTES:\n {data['position'].value_counts()}")

    return data


In [ ]:
data = tagWParms(data_frame, Lwindow=4380, Swindow=90)

In [ ]:
def tagByWindow(df, i, threshold, signal, window):
    
    df_size = df['Close'].size
    actualValue = df.loc[i,'Close']
    #derecha
    j = i + window
    while(j <= lastVI and abs(df.loc[j,'Close'] - actualValue) <= threshold):
      df.loc[j,'position'] = signal
      j += d
    #izquierda
    k = i - d
    while(k >= firstVI and abs(df.loc[k,'Close'] - actualValue) <= threshold):
      df.loc[k,'position'] = signal
      k -= d
    i = j- d    
    return df,j



In [ ]:
"""
plt.plot(data['Close'].loc[date_rng], label='Close Price MXN', lw=1)
plt.plot(data.loc[date_rng][data['position']== -1]['Close'],'v', color='orange', fillstyle='full',label='Sell Signal')
plt.plot(data.loc[date_rng][data['position'] == 1]['Close'],'^', color='green', fillstyle='full',label='Buy Signal')
plt.plot(data.loc[date_rng][data['position']== -2]['Close'],'v', color='red', fillstyle='full',label='Strong Sell Signal')
plt.plot(data.loc[date_rng][data['position'] == 2]['Close'],'^', color='blue', fillstyle='full',label='Strong Buy Signal')

plt.xlabel('date')
plt.ylabel('BITCOIN TO MXN')
plt.grid(True)
plt.legend(loc=1)
plt.setp(plt.gca().get_xticklabels(), rotation = 30)
"""

# Create random data with numpy

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=data.index, y=data['Close'],
                    mode='lines',
                    name='Close'))

fig.add_trace(go.Scatter(x=data[data['position'] == -2].index, y=data[data['position'] == -2]['Close'],
                    mode='markers',
                    name='Top Sale'))
fig.add_trace(go.Scatter(x=data[data['position'] == -1].index, y=data[data['position'] == -1]['Close'],
                    mode='markers',
                    name='Sale'))
fig.add_trace(go.Scatter(x=data[data['position'] == 1].index, y=data[data['position'] == 1]['Close'],
                    mode='markers',
                    name='Buy'))
fig.add_trace(go.Scatter(x=data[data['position'] == 2].index, y=data[data['position'] == 2]['Close'],
                    mode='markers',
                    name='Top Buy'))


fig.show()


# Pasar al backtesting
Tomando el etiquetado con diferente ventana Hacer resampling y mandar a backtest

In [ ]:
data['Close delta'] = data['Close'].diff()

In [ ]:
date_rng = pd.date_range(start='2017-08-17 04:00:00', end='2017-08-18 00:00:00', freq='1H')
date_rng

In [ ]:
data.loc[date_rng]

In [ ]:
#data.loc['2017-08-17 04:00:00']
data.loc[date_rng]
#data.min()

In [ ]:
data.min()

In [ ]:
date_rng = pd.date_range(start='2017-12-22 19:00:00', end='2017-12-22 21:00:00', freq='1H')
date_rng

In [ ]:
data.loc[date_rng]

In [ ]:
data.max()

In [ ]:
data[data['Close delta'] == -1577.87]

In [ ]:
data.loc['2017-12-22 20:00:00']

In [ ]:
data = etiq(data, threshold=.3999)

In [ ]:
print(f"VALORES DESPUES:\n {data['position'].value_counts()}")

In [ ]:
#print(data.head(10))
#plotDataset(data)
#data = data.drop(columns=['mn', 'mx'])
#data.to_csv('/content/gdrive/My Drive/Colab Notebooks/BITSO/data/datasetEtiquetado.csv')

In [ ]:
data.head()

In [ ]:
len(data.index)

In [ ]:
date_rng = pd.date_range(start='2017-08-17 04:00:00', end='2017-12-23 12:00:00', freq='1H')
date_rng

In [ ]:
# Create random data with numpy
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=data.index, y=data['Close'],
                    mode='lines',
                    name='Close'))
#fig.add_trace(go.Scatter(x=random_x, y=random_y1,
#                    mode='lines+markers',
#                    name='lines+markers'))

fig.add_trace(go.Scatter(x=data[data['position'] == -2].index, y=data[data['position'] == -2]['Close'],
                    mode='markers',
                    name='Top Sale'))
fig.add_trace(go.Scatter(x=data[data['position'] == -1].index, y=data[data['position'] == -1]['Close'],
                    mode='markers',
                    name='Sale'))
fig.add_trace(go.Scatter(x=data[data['position'] == 1].index, y=data[data['position'] == 1]['Close'],
                    mode='markers',
                    name='Buy'))
fig.add_trace(go.Scatter(x=data[data['position'] == 2].index, y=data[data['position'] == 2]['Close'],
                    mode='markers',
                    name='Top Buy'))

fig.show()

In [ ]:
#fig = px.histogram(data.loc[date_rng], x="Close", marginal = "rug",nbins=10)
fig = px.histogram(data, x="Close", marginal = "rug",nbins=10)
fig.show()

# TODO
* Considerar Estrategia de trading
* jugar con ventanas que se ajusten a la frecuencia de operaciones
* Ver el futuro usando la ventana correspondiente al horizonte de la estrategia
* Considerar la comisión
* La ventana es un parámetro a configurar
* es decir N > n ventana grande vs ventana pequeña y etiquetar en base a ese conocimiento

**Hacer análisi de dataset
calcular del dataset el número de eventos a tomar
cuanta información para atrás necesito ver para la predicción que se quiere hacer**

Una técnica es la autocorrelación 
Hacer resampling para 1 hora cada 5 horas cada día cada semana etc...

In [ ]:
# pick and test
def pickByRegion(df, window):
    

    df_size = df['Close'].size

    actualValue = df.loc[i,'Close']

    loops = df_size / window

    #derecha

    i = firstVI + d
    curPos = df.loc[j,'position']
    while(i <= lastVI):    
        if(abs(df.loc[j,'position']) != 2):
          df.loc[j,'position'] = signal
        i += d
    
    #izquierda
    k = i - d
    while(k >= firstVI and abs(df.loc[k,'Close'] - actualValue) <= threshold):    
        if(abs(df.loc[k,'position']) != 2):      
          df.loc[k,'position'] = signal
        k -= d
    i = j- d    
    return df


In [ ]:
"""register_matplotlib_converters()

fig = plt.figure(figsize=[15,10])

#grafico superior
#ax = fig.add_subplot(2, 1, 1)
#ax.set_xticklabels([])
plt.plot(data['Close'].loc[date_rng], label='Close Price MXN', lw=1)
plt.plot(data['SMA1'].loc[date_rng], fillstyle='none',label='SMA1', lw=1)
plt.plot(data['SMA2'].loc[date_rng],fillstyle='none',label='SMA2', lw=1)
plt.plot(data['SMAEnter'].loc[date_rng],fillstyle='none',label='SMA Enter', lw=1)
plt.plot(data['SMAExit'].loc[date_rng],fillstyle='none',label='SMA Exit', lw=1)
plt.plot(data.loc[date_rng][data['position']== -1]['Close'],'v', color='orange', fillstyle='full',label='Sell Signal')
plt.plot(data.loc[date_rng][data['position'] == 1]['Close'],'^', color='green', fillstyle='full',label='Buy Signal')
plt.plot(data.loc[date_rng][data['position']== -2]['Close'],'v', color='red', fillstyle='full',label='Sell Signal')
plt.plot(data.loc[date_rng][data['position'] == 2]['Close'],'^', color='blue', fillstyle='full',label='Buy Signal')

plt.xlabel('date')
plt.ylabel('BITCOIN TO MXN')
plt.grid(True)
plt.legend(loc=4)
plt.setp(plt.gca().get_xticklabels(), rotation = 30)
"""

In [ ]:
date_rng = pd.date_range(start='2017-12-15 00:00:00', end='2017-12-23 12:00:00', freq='H')
date_rng

Decomposition of the Time Series into Systematic and Non-systematic components.

* Systematic: Components of the TIme Series have consistensy and/or Recurrence and can be described and modeled.
* Non-Systematic: Components of the time series that can not be directly modeled.

A given time series is thought to consist of three systematic components including level, trend, seasonability , and one non-systematic component called noise.

These components are described as follows:
* **Level**: The average value in the series
* **Trend**: The increasing or decreasing value in the series
* **Seasonality**: The repeating short-term cycle in the series.
* **Noise**: The random variation in the series.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
type(data['Close'])

In [ ]:
series = data['Close']

In [ ]:

result = seasonal_decompose(series, model='additive')
print(result.trend)
print(result.seasonal)
print(result.resid)
print(result.observed)


In [ ]:
from pandas.plotting import autocorrelation_plot
plt.figure(figsize=(15,10))
autocorrelation_plot(series[:50])

In [ ]:
  gMax = data['Close'].max()
  gMin = data['Close'].min()
  print(f"{gMax} : {gMin}")

data['quantile_5'] = pd.qcut(data['Close'], q=5)

In [ ]:
data['quantile_5'].value_counts()

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=data.index, y=data['Close'],
                    mode='lines',
                    name='Close'))
#fig.add_trace(go.Scatter(x=random_x, y=random_y1,
#                    mode='lines+markers',
#                    name='lines+markers'))
fig.add_trace(go.Scatter(x=data[data['position'] == -2].index, y=data[data['position'] == -2]['Close'],
                    mode='markers',
                    name='Top Sale'))
fig.add_trace(go.Scatter(x=data[data['position'] == -1].index, y=data[data['position'] == -1]['Close'],
                    mode='markers',
                    name='Sale'))
fig.add_trace(go.Scatter(x=data[data['position'] == 1].index, y=data[data['position'] == 1]['Close'],
                    mode='markers',
                    name='Buy'))
fig.add_trace(go.Scatter(x=data[data['position'] == 2].index, y=data[data['position'] == 2]['Close'],
                    mode='markers',
                    name='Top Buy'))
"""
fig.add_shape(
        # Line Horizontal
            type="line",
            x0=firstVI,
            y0=77881.83,
            x1=lastVI,
            y1=77881.83,
            line=dict(
                color="LightSeaGreen",
                width=1,
                dash="dashdot",
            ),
    )
fig.add_shape(
        # Line Horizontal
            type="line",
            x0=firstVI,
            y0=151200.0,
            x1=lastVI,
            y1=151200.0,
            line=dict(
                color="LightSeaGreen",
                width=1,
                dash="dashdot",
            ),
    )
fig.add_shape(
        # Line Horizontal
            type="line",
            x0=firstVI,
            y0=124689.248,
            x1=lastVI,
            y1=124689.248,
            line=dict(
                color="LightSeaGreen",
                width=1,
                dash="dashdot",
            ),
    )
fig.add_shape(
        # Line Horizontal
            type="line",
            x0=firstVI,
            y0=180497.16,
            x1=lastVI,
            y1=180497.16,
            line=dict(
                color="LightSeaGreen",
                width=1,
                dash="dashdot",
            ),
    )

fig.add_shape(
        # Line Horizontal
            type="line",
            x0=firstVI,
            y0=397998.76,
            x1=lastVI,
            y1=397998.76,
            line=dict(
                color="LightSeaGreen",
                width=1,
                dash="dashdot",
            ),
    )
"""
fig.show()

In [ ]:
data

In [ ]:
len(data)

In [ ]:
positions = data['position']

Backtesting.py documentation
https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html


***Crear columna de posicion original y ahí dejar el etiquetado original de argrelextrema antes de ninguna rtansformacion. ESP funcion tagForOptimumSales
Esto nos va apermitir comparar en posiciones de pérdida las transformaciones de esa posición !!!!!!***

In [ ]:
########################################################################################
###### Implementacion de estrategia con señal generada a partir de argrelextrema #######
########################################################################################

dataN = pd.DataFrame()
from backtesting.lib import SignalStrategy, TrailingStrategy

class Agrelextrema(SignalStrategy,TrailingStrategy):
    #n1 = 720
    #n2 = 8760
    n1 = 90
    #n1 = 360
    n2= 4380
    commission = 0.0085

    d = timedelta(hours=1)
    #d = timedelta(minutes=5)

    """
    def merge(self, data1,data2):
        df = data1.copy()
        for i in df.index:
            value = data2.loc[i,'position']
            if value != 0:
                df.loc[i,'position'] = value
        return df
    """


    def merge(self, data1, data2):
        dfp1 = data1['position']
        dfp2 = data2['position']
        dfp = dfp2.combine(dfp1, (lambda x1, x2: x1 if x1 != 0 else x2), fill_value=0)
        return dfp


    def convertMax(self, df,i,threshold,signal, comission):
        
        df_size = df['Close'].size
        actualValue = df.loc[i,'Close']

        #Señal de venta
        if (signal != 0):
          #derecha
          j = i + self.d
          #while(j <= lastVI and abs(df.loc[j,'Close'] - actualValue) <= threshold and abs(df.loc[j,'Close'] - actualValue) > comission):
          while(j <= lastVI and abs(df.loc[j,'Close'] - actualValue) <= threshold and abs(df.loc[j,'Close'] - actualValue) > comission):        
            df.loc[j,'position'] = signal
            j += self.d
          #izquierda
          #NO HACR IZQUIERDA
          """k = i - self.d
          while(k >= firstVI and abs(df.loc[k,'Close'] - actualValue) <= threshold and abs(actualValue - df.loc[k,'Close']) > comission):
            df.loc[k,'position'] = signal
            k -= self.d
          i = j- self.d"""    

        """
        #Señal de compra
        if (signal < 0 ):
          #derecha
          j = i + self.d
          while(j <= lastVI and abs(df.loc[j,'Close'] - actualValue) <= threshold and abs(df.loc[j,'Close'] - actualValue) > comission):
            df.loc[j,'position'] = signal
            j += d
        """    
          #izquierda
          #NO HACER IZQUIERDA          
        """k = i - d
        while(k >= firstVI and abs(df.loc[k,'Close'] - actualValue) <= threshold and abs(actualValue - df.loc[k,'Close']) > comission):    
          df.loc[k,'position'] = signal
          k -= self.d
        i = j- self.d"""

        return df,j

    """
    def etiq(self, df, threshold=.15, comission=.042):
        #print(f"Etiquetando:")
        #df_size = df['Close'].size
        i = firstVI
        while i < lastVI:
            signal = data.loc[i,'position']
            if signal == 2 or signal == -2:
                closeValue = df.loc[i,'Close']
                df,i = self.convertMax(df,i,(closeValue * threshold),signal, comission) #.399 is the highest delta for Nov 2017 through Jan 2018 highest variance
            i += d
        i = firstVI
        
        while i < lastVI:
            signal = data.loc[i,'position']
            if signal == 1 or signal == -1:
                closeValue = df.loc[i,'Close']
                df,i = self.convertMax(df,i,(closeValue * threshold),signal, comission)
            i += d
        
        return df
    """

    def etiq(self, df, threshold=.15, comission=.042):
        #print(f"Etiquetando:")
        #df_size = df['Close'].size
        i = firstVI
        for idx, row in data.iterrows():
            signal = row['position']
            if signal != 0:
                closeValue = row['Close']
                df,i = self.convertMax(df,i,(closeValue * threshold),signal, comission) #.399 is the highest delta for Nov 2017 through Jan 2018 highest variance

        
        return df            


    def previousPosition(self, df, i):
      if i > firstVI :
        prev = i - self.d
      else: prev = i

      while (prev > firstVI and df.loc[prev, 'position'] == 0):
        prev = prev - self.d
      return prev

    def tagForOptimumSales(self, df, commission):
      #i = firstVI
      movs = 0
      df1 = df.copy(deep=True)


      prevPos = df1.loc[firstVI, 'position']
      pos=df1.loc[firstVI, 'position']
      #bestSaleIdx = firstVI
      #bestBuyIdx =  firstVI
      #prevPrice = df1.loc[firstVI, 'Close']
      prev = firstVI
      lastSaleIdx = 0
      lastBuyIdx = 0

      for i, row in df1.iterrows():
        #revisar la posicion

        #obtener la posición de compra
        #si el movimiento anterior fue compra la venta debe de tener ganancia
        #si el movimiento anterior fue venta el movimiento actual debve tener más ganancia que aquel y se suprime la venta anterior
        #TODO:
        #cuando posiciones adyacentes son diferentes, intentar intercambiar las posiciones en vez de eliminar la peor compra!!!
        ##############################
        if row['position'] != 0: #solo para posiciones de compra y venta.
          pos = row['position']
          prev = self.previousPosition(df, i)
          prevPos = df1.loc[prev, 'position']

          #if (prevPos > 0 and pos < 0) or (prevPos < 0 and pos > 0): #cambiamos de posición. Revisar este if . puede estar mal
          if (prevPos != pos ): #cambiamos de posición.
            if prevPos > 0 and pos < 0 : #compra y luego venta
              if df1.loc[prev, 'Close'] > df1.loc[i, 'Close']:  #intercambiar posiciones
                if (df1.loc[prev, 'Close'] - df1.loc[i, 'Close']) > (df1.loc[i, 'Close'] * commission): #si le ganamos a la comisión            
                  df1.loc[prev, 'position'] = -2.0 #cancela compra
                  df1.loc[i, 'position'] = 2.0 #cancela compra
                  df1.loc[prev, 'transformPath'] = 'CVswap'
                  df1.loc[i, 'transformPath'] = 'CVswap'
                else: #quita compra
                  df1.loc[prev, 'position'] = 0.0 #cancela venta
                  df1.loc[i, 'position'] = 0.0 #cancela venta
                  df1.loc[prev, 'transformPath'] = 'CVcancel'
                  df1.loc[i, 'transformPath'] = 'CVcancel'
                  
                movs += 1
                #print("Cancel buy at " + str(i))
            if prevPos < 0 and pos > 0 : #venta y luego compra
              if df1.loc[prev, 'Close'] <= df1.loc[i, 'Close']:  #intercambiar posiciones
                if (df1.loc[i, 'Close'] - df1.loc[prev, 'Close']) > (df1.loc[i, 'Close'] * commission): #si le ganamos a la comisión
                  df1.loc[i, 'position'] = -2.0 #cancela compra
                  df1.loc[prev, 'position'] = 2.0 #cancela compra
                  df1.loc[prev, 'transformPath'] = 'CVcancelC'
                  df1.loc[i, 'transformPath'] = 'CVcancelC'

                else: #quita la compra
                  df1.loc[prev, 'position'] = 0.0 #cancela venta
                  df1.loc[i, 'position'] = 0.0 #cancela venta
                  df1.loc[prev, 'transformPath'] = 'CVcancelV'
                  df1.loc[i, 'transformPath'] = 'CVcancelV'

                movs += 1
            
            #Si Venta Anterior es mayor que Compra Actual no se hace nada
            pos = df1.loc[i, 'position']
          else: #seguimos en la misma posicion 
            #if pos > 0 and (df1.loc[i, 'Close'] - df1.loc[prev, 'Close']) >= (commission * df1.loc[i, 'Close']) : #venta compuesto
            if pos > 0 and (df1.loc[i, 'Close'] >= df1.loc[prev, 'Close']): #compra
              #df1.loc[prev, 'position'] = 0 #quitar la posicion de la venta previa
              df1.loc[i, 'position'] = 0 #quitar la posicion de la compra actual
              df1.loc[i, 'transformPath'] = 'CCcancelC2'
              movs += 1
              #if (df1.loc[i, 'Close'] > df1.loc[bestSaleIdx, 'Close']):
                #bestSaleIdx = i #posicion actual es la mejor venta
              
            #elif pos > 0 and (df1.loc[i, 'Close'] - df1.loc[prev, 'Close']) < (commission * df1.loc[i, 'Close']) : #venta            
            elif pos > 0 and (df1.loc[i, 'Close'] < df1.loc[prev, 'Close']) : #compra           AQUI HAY QUE REVISAR DE NUEVO CONTRA LA POSICION ANTERIOR!!!!!!!!!!!!!!! 
              df1.loc[prev, 'position'] = 0 #se elimina esta compra al no ser mejor
              df1.loc[prev, 'transformPath'] = 'CCcancelC1'

              movs += 1 
              #print("Cancel sell at " + str(i))
            
            elif pos < 0 :
              if (df1.loc[i, 'Close'] >= df1.loc[prev, 'Close']): #venta
                df1.loc[prev, 'position'] = 0 #quitar la posicion de la venta previa
                df1.loc[prev, 'transformPath'] = 'VVcancelV1'                
                movs += 1
                #if (df1.loc[i, 'Close'] > df1.loc[bestSaleIdx, 'Close']):
                  #bestSaleIdx = i #posicion actual es la mejor venta
                #print("Cancel sell at " + str(prev))
              if (df1.loc[i, 'Close'] < df1.loc[prev, 'Close']): #venta
                df1.loc[i, 'position'] = 0 #quitar la posicion de la venta previa
                df1.loc[i, 'transformPath'] = 'VVcancelV2'
                movs += 1

              #lastBuy = i

          #prev = i #ultima operacion de cada loop hacer movimiento actual el previo
          
        #print("moves "+ str(movs))
      
      return df1  



        #si es compra comparar con la compra existente, si es mejor, reemplaza a (best_buy) de los contrario nos quedamos con esta última

        #si es venta revisar ganancias usando la mejor venta almacenada si es buena se queda si la ganancia no cubre la comisión mas una ganancia se convierte en "no hacer nada [posiciaon 0]"
        #si la venta mejor cambió, hay que moificar la venta buena original a 0 (opcional)


    def tagWParms(self, df , Lwindow, Swindow): 

        df1 = pd.DataFrame()

        df1['Open'] = self.data.Open.to_series()
        df1['High'] = self.data.High.to_series()
        df1['Low'] = self.data.Low.to_series()
        df1['Close'] = self.data.Close.to_series()
        #df1 = df.copy(deep=True)

        dataMinor = df1.copy()
        dataMax = df1.copy()    
        #procesando ventas menores
        #n = Swindow #un mes   
        dataMinor['mn'] = dataMinor.iloc[argrelextrema(dataMinor.Close.values, np.less_equal, order=Swindow)[0]]['Close']
        dataMinor['mx'] = dataMinor.iloc[argrelextrema(dataMinor.Close.values, np.greater_equal, order=Swindow)[0]]['Close']
        dataMinor['position'] = 0
        dataMinor.loc[dataMinor['mn'].notnull(),'position'] = 1.0 
        dataMinor.loc[dataMinor['mx'].notnull(),'position'] = -1.0 

        #n = Lwindow #un año
        dataMax['mn'] = dataMax.iloc[argrelextrema(dataMax.Close.values, np.less_equal, order=Lwindow)[0]]['Close']
        dataMax['mx'] = dataMax.iloc[argrelextrema(dataMax.Close.values, np.greater_equal, order=Lwindow)[0]]['Close']
        dataMax['position'] = 0
        dataMax.loc[dataMax['mn'].notnull(),'position'] = 2.0 
        dataMax.loc[dataMax['mx'].notnull(),'position'] = -2.0 

        pos = self.merge(dataMinor,dataMax)
        data['position'] = pos
        #print(f"VALORES ANTES:\n {dataN['position'].value_counts()}")

        return data
      



    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize all the classes
        #print("Windows Range: %i - %i"%(self.n1, self.n2))
        global dataN
        super().init()
        

        dataN = self.tagWParms(self.data, self.n2, self.n1)        
        #dataN = self.etiq(dataN, threshold=.3999)
        #print(f"VALORES DESPUES:\n {dataN['position'].value_counts()}")
        #dataN = dataN.drop(columns=['mn', 'mx'])
        dataN['positionO'] = dataN['position']
        dataN['transformPath'] = ''
        dataN = self.tagForOptimumSales(dataN, commission = self.commission) #check this part
        

        self.set_signal(pd.Series(dataN['position']),plot=True)
        #data.to_csv('/content/gdrive/My Drive/Colab Notebooks/BITSO/data/datasetEtiquetado.csv')        


In [ ]:
data

In [ ]:
databt = data.copy()

In [ ]:
databt

In [ ]:
databt = databt.drop(columns=['Close delta', 'quantile_5','position'], axis=1)

In [ ]:
databt.loc['2019-11-05 20:00:00']

In [ ]:

bt1 = Backtest(databt, Agrelextrema, cash=20000, commission=.0065, trade_on_close=True) 

In [ ]:

pdBtr = bt1.run()

In [ ]:
pdBtr

* An long entry signal is considered present wherever entry is greater than zero.(compra) symbol = v
* A short entry signal is considered present wherever entry is less than zero. (venta) symbol = ^
* If exit is provided, a nonzero value closes the position, if any; otherwise the position is held until a reverse signal in entry.
If plot is True, the signal entry/exit indicators are plotted when Backtest.plot() is called.


In [ ]:
#^ = venta
#v = compra
#plt.clf()
bt1.plot()  

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=dataN.index, y=dataN['Close'],
                    mode='lines',
                    name='Close'))

fig.add_trace(go.Scatter(x=dataN[dataN['position'] == -2].index, y=dataN[dataN['position'] == -2]['Close'],
                    mode='markers',
                    marker = dict(size=12, color= 'blue',symbol= 'star-triangle-up'),
                    name='Top Sale'))
fig.add_trace(go.Scatter(x=dataN[dataN['position'] == -1].index, y=dataN[dataN['position'] == -1]['Close'],
                    mode='markers',
                    marker = dict(size=12,color= 'green',symbol= 'star-triangle-up' ),                    
                    name='Sale'))
fig.add_trace(go.Scatter(x=dataN[dataN['position'] == 1].index, y=dataN[dataN['position'] == 1]['Close'],
                    mode='markers',
                    marker = dict(size=12,color= 'orange',symbol= 'star-triangle-down'),                    
                    name='Buy'))
fig.add_trace(go.Scatter(x=dataN[dataN['position'] == 2].index, y=dataN[dataN['position'] == 2]['Close'],
                    mode='markers',
                    marker = dict(size=12,color= 'red',symbol= 'star-triangle-down' ),                    
                    name='Top Buy'))

fig.show()

In [ ]:
dataN[dataN['position'] > 0]

In [ ]:
dataN.to_csv('/content/gdrive/My Drive/Colab Notebooks/BINANCE/data/Binance_BTCUSDT_1h_labeled_2_windows.csv')

In [ ]:
dataN[dataN['position'] < 0]

In [ ]:
date_rng = pd.date_range(start='2017-12-22 19:00:00', end='2017-12-22 21:00:00', freq='H')
dataN.loc['2019-10-28 01:00:00']

In [ ]:
dataN.loc['2019-11-05 20:00:00']

In [ ]:
dataN.loc['2019-11-25 05:00:00']

In [ ]:
dataN.loc['2019-11-29 15:00:00']

In [ ]:
dataN['position'].value_counts()

Optimizando Return

In [ ]:
stats, heatmap = bt1.optimize(n1 = [1,2,4,6,10,24,45,90,180,360,720,1440],
                  n2 = [5,9,18,35,69,138,274,548,1095,2190,4380,8760],
                  constraint=lambda p: p.n2 > p.n1,
                  #maximize='Win Rate [%]',
                  maximize='Return [%]',
                  return_heatmap=True
                  )

In [ ]:
#bt1.data.Close[]

In [ ]:
stats

In [ ]:
heatmap

In [ ]:
hm = heatmap.groupby(['n1', 'n2']).mean().unstack()
hm

In [ ]:
import seaborn as sns
from backtesting.lib import plot_heatmaps
plt.figure(figsize=(15,10))

#sns.heatmap(hm[::-1], cmap='viridis')
plot_heatmaps(heatmap, agg='mean')

Optimizando Win Rate

In [ ]:
bt2 = Backtest(databt, Agrelextrema, cash=20000, commission=.0065, trade_on_close=True) 

In [ ]:
bt2.run()

In [ ]:
stats2, heatmap2 = bt2.optimize(n1 = [1,2,4,6,12,24,45,90,180,360,720,1440],
                  n2 = [5,9,18,35,69,138,274,548,1095,2190,4380,8760],
                  constraint=lambda p: p.n2 > p.n1,
                  #maximize='Win Rate [%]',
                  maximize='Win Rate [%]',
                  return_heatmap=True
                  )

In [ ]:
stats2

In [ ]:
heatmap2

In [ ]:
hm2 = heatmap2.groupby(['n1', 'n2']).mean().unstack()
hm2

In [ ]:
plt.figure(figsize=(15,10))
#sns.heatmap(hm2[::-1], cmap='viridis')
plot_heatmaps(heatmap2, agg='mean')

## Agrelextrema con una sola ventana

#  Revisar para eliminar posisiones  de -1 (venta consecutivas)

In [ ]:
########################################################################################
###### Implementacion de estrategia con señal generada a partir de argrelextrema #######
########################################################################################

dataN = pd.DataFrame()

from backtesting.lib import SignalStrategy, TrailingStrategy

class Agrelextrema1W(SignalStrategy,TrailingStrategy):
    #n1 = 13
    n1 = 14
    n2= 0
    commission = 0.0085

    d = timedelta(hours=1)

    def previousPosition(self, df, i):
      if i > firstVI :
        prev = i - self.d
      else: prev = i

      while (prev > firstVI and df.loc[prev, 'position'] == 0):
        prev = prev - self.d
      return prev

    def nextPosition(self, df, i):
      if i < lastVI :
        nxt = i + self.d
      else: nxt = i

      while (nxt < lastVI and df.loc[nxt, 'position'] == 0):
        nxt = nxt + self.d
      return nxt


    def merge(self, data1,data2):
        df = data1.copy()
        for i in df.index:
            value = data2.loc[i,'position']
            if value != 0:
                df.loc[i,'position'] = value
        return df

    #Obtiene la mejor y la peor posicion de un bloque
    def getBestWorstPos(self, dfBlock):

      #print(dfBlock)

      if (dfBlock.position.min() > 0 ): #Then compra
        best = dfBlock.Close.min()
        worst = dfBlock.Close.max()
      else: #then venta
        best = dfBlock.Close.max()
        worst = dfBlock.Close.min()

      return best, worst


    def convertMax(self, df,i,thresholdPct,signal, comission): #Propagar sólo hacia la derecha
        
        df_size = df['Close'].size
        actualValue = df.loc[i,'Close']
        prevPosIdx = self.previousPosition(df, i) #commented for testing purposes
        nextPosIdx = self.nextPosition(df, i)
        positionBlockDf = df[df['position_block'] == prevPosIdx]

        #print(positionBlockDf)

        prevBest, prevWorst = self.getBestWorstPos(positionBlockDf) #Estos son los valores con los que se van a hacer las comparaciones para los deltas (las peores)
        #print(i,':',df.loc[i, 'position'],' C:',df.loc[i, 'Close'],' pB:',prevBest, ' pW:', prevWorst)

        #Señal de venta
        if (signal < 0):
          #derecha
          j = i + self.d
          #while(j <= lastVI and abs(df.loc[j,'Close'] - actualValue) <= threshold and abs(df.loc[j,'Close'] - actualValue) > comission):
          while((j <= lastVI ) 
                and (abs( actualValue - df.loc[j,'Close']) / df.loc[j,'Close']  <= thresholdPct) 
                and (abs( df.loc[j,'Close'] - prevBest ) > df.loc[j,'Close'] * comission) 
                and (abs(df.loc[j,'Close']) > abs(prevBest))
                and (df.loc[j, 'propagated_position'] == 0)
                and (abs(df.loc[nextPosIdx, 'Close'] - df.loc[j, 'Close']) > df.loc[nextPosIdx, 'Close'] * comission ) #tenemos que ganar a la comision en el siguiente
            ):        
            df.loc[j,'propagated_position'] = signal
            df.loc[j,'position_block'] = i
            df.loc[j,'position_close'] = actualValue
            df.loc[j,'position_block_delta_pct'] = abs(actualValue - df.loc[j,'Close']) / df.loc[j,'Close']
            j += self.d
          #izquierda
          #NO HACER IZQUIERDA
          """k = i - self.d
          while((k >= firstVI and abs(actualValue / df.loc[k,'Close'] ) <= threshold and abs(df.loc[k,'Close'] - actualValue) > df.loc[k,'Close'] * comission) and df.loc[k, 'propagated_position'] == 0):
            df.loc[k,'propagated_position'] = signal
            df.loc[j,'position_block'] = i
            df.loc[j,'position_close'] = actualValue

            k -= self.d """
          
          #Señal de compra
        
        if (signal > 0 ):
          #derecha
          j = i + self.d
          while((j <= lastVI ) 
                and (abs( df.loc[j,'Close'] - actualValue )/ df.loc[j,'Close']  <= thresholdPct) 
                and (abs(prevWorst - df.loc[j,'Close']) > df.loc[j,'Close'] * comission) 
                and (df.loc[j,'Close'] < prevWorst)
                and (df.loc[j, 'propagated_position'] == 0)
                and (abs(df.loc[j, 'Close'] - df.loc[nextPosIdx, 'Close']) > df.loc[nextPosIdx, 'Close'] * comission ) #tenemos que ganar a la comision en el siguiente
            ):
            df.loc[j,'propagated_position'] = signal
            df.loc[j,'position_block'] = i
            df.loc[j,'position_close'] = actualValue
            df.loc[j,'position_block_delta_pct'] = abs( df.loc[j,'Close'] - actualValue ) / df.loc[j,'Close']                         
            j += self.d

          #izquierda
          #NO HACER IZQUIERDA
          """k = i - self.d
          while((k >= firstVI and abs(actualValue / df.loc[k,'Close'] ) <= threshold and abs(df.loc[k,'Close'] - actualValue) > df.loc[k,'Close'] * comission) and df.loc[k, 'propagated_position'] == 0):
            df.loc[k,'propagated_position'] = signal
            df.loc[j,'position_block'] = i
            df.loc[j,'position_close'] = actualValue

            k -= self.d"""
          #i = j- self.d

        return df,j


    def etiq(self, df, threshold=0.25, comission=0.0065):
        #print(f"Etiquetando:")
        df_size = df['Close'].size
        
        df1 = df.copy(deep=True)
        df1['propagated_position'] = df1['position']
        df1['position_block'] = df1.index
        df1['position_close'] = df1['Close']
        df1['position_block_delta_pct'] = 0

        i = firstVI
        while i < lastVI:
            signal = df1.loc[i,'propagated_position']
            if signal != 0:
                closeValue = df1.loc[i,'Close']
                #df,i = self.convertMax(df1,i,(closeValue * threshold),signal, comission) #.399 is the highest delta for Nov 2017 through Jan 2018 highest variance
                df,i = self.convertMax(df1,i,(threshold),signal, comission) #changing threshold criteria to pure threshold (no calculation)
            i += d        
        
        return df1
    
    
    def tagForOptimumSales(self, df, commission):
      #i = firstVI
      movs = 0
      df1 = df.copy(deep=True)


      prevPos = df1.loc[firstVI, 'position']
      pos=df1.loc[firstVI, 'position']
      #bestSaleIdx = firstVI
      #bestBuyIdx =  firstVI
      prevPrice = df1.loc[firstVI, 'Close']
      prev = firstVI
      lastSaleIdx = 0
      lastBuyIdx = 0

      for i in df1.index:
        #revisar la posicion

        #Si el movimiento es venta
        #si el movimiento anterior fue compra la venta debe de tener ganancia
        #si el movimiento anterior fue venta el movimiento actual debve tener más ganancia que aquel y se suprime la venta anterior
        #Si el movimiento es compra
        #Si el movimiento anterior fue compra se suplrime la peor compra y se queda sólo la mejor
        #Si el movimiento anterior fue venta se debe tener ganancia. SI no hay ganancia se cambian de lugar o se suprime la compra (o ambos)
        #TODO:
        #cuando posiciones adyacentes son diferentes, intentar intercambiar las posiciones en vez de eliminar la peor compra!!!
        ##############################
        if df1.loc[i, 'position'] != 0: #solo para posiciones de compra y venta.
          pos = df1.loc[i, 'position']
          prev = self.previousPosition(df, i)
          prevPos = df1.loc[prev, 'position']

          if (prevPos > 0 and pos < 0) or (prevPos < 0 and pos > 0): #cambiamos de posición. Revisar este if . puede estar mal
            if prevPos > 0 and pos < 0 : #compra y luego venta
              if df1.loc[prev, 'Close'] > df1.loc[i, 'Close']:  #intercambiar posiciones
                if (abs(df1.loc[prev, 'Close'] - df1.loc[i, 'Close'])) > abs(df1.loc[i, 'Close'] * commission): #si le ganamos a la comisión            
                  df1.loc[prev, 'position'] = -2.0 #intercambia compra
                  df1.loc[i, 'position'] = 2.0 #intercambia venta
                else: #quita compra y venta
                  df1.loc[prev, 'position'] = 0.0 #cancela compra
                  df1.loc[i, 'position'] = 0.0 #cancela venta
              else:
                if (abs(df1.loc[prev, 'Close'] - df1.loc[i, 'Close'])) < abs(df1.loc[i, 'Close'] * commission): #si le ganamos a la comisión            
                  df1.loc[prev, 'position'] = -2.0 #cancela compra
                  df1.loc[i, 'position'] = 2.0 #cancela compra
              movs += 1
                #print("Cancel buy at " + str(i))
            if prevPos < 0 and pos > 0 : #venta y luego compra
              if df1.loc[prev, 'Close'] <= df1.loc[i, 'Close']:  #intercambiar posiciones
                if (abs(df1.loc[i, 'Close'] - df1.loc[prev, 'Close'])) > abs(df1.loc[i, 'Close'] * commission): #si le ganamos a la comisión
                  df1.loc[i, 'position'] = -2.0 #mantiene compra
                  df1.loc[prev, 'position'] = 2.0 #mantiene compra
                else: #quita la compra
                  df1.loc[prev, 'position'] = 0.0 #cancela venta
                  df1.loc[i, 'position'] = 0.0 #cancela venta
                movs += 1
            
            #Si Venta Anterior es mayor que Compra Actual no se hace nada
            pos = df1.loc[i, 'position']
          else: #seguimos en la misma posicion 
            #if pos > 0 and (df1.loc[i, 'Close'] - df1.loc[prev, 'Close']) >= (commission * df1.loc[i, 'Close']) : #venta compuesto
            if pos > 0 and (df1.loc[i, 'Close'] >= df1.loc[prev, 'Close']): #compra
              #df1.loc[prev, 'position'] = 0 #quitar la posicion de la venta previa
              df1.loc[i, 'position'] = 0 #quitar la posicion de la compra actual
              movs += 1
              
            #elif pos > 0 and (df1.loc[i, 'Close'] - df1.loc[prev, 'Close']) < (commission * df1.loc[i, 'Close']) : #venta            
            elif pos > 0 and (df1.loc[i, 'Close'] < df1.loc[prev, 'Close']) : #compra            
              df1.loc[prev, 'position'] = 0 #se elimina esta compra al no ser mejor
              movs += 1 
              #print("Cancel sell at " + str(i))
            
            if pos < 0 : #ANtes era elif revisar todo mañana con calma 07/02/2020
              if (df1.loc[i, 'Close'] >= df1.loc[prev, 'Close']): #venta
                df1.loc[prev, 'position'] = 0 #quitar la posicion de la venta previa
                movs += 1
                #print("Cancel sell at " + str(prev))
              if (df1.loc[i, 'Close'] < df1.loc[prev, 'Close']): #venta
                df1.loc[i, 'position'] = 0 #quitar la posicion de la venta previa
                movs += 1

              lastBuy = i

          #prev = previousPosition(df, i) #ultima operacion de cada loop hacer movimiento actual el previo
          
        #print("moves "+ str(movs))
      
      return df1  



        #si es compra comparar con la compra existente, si es mejor, reemplaza a (best_buy) de los contrario nos quedamos con esta última

        #si es venta revisar ganancias usando la mejor venta almacenada si es buena se queda si la ganancia no cubre la comisión mas una ganancia se convierte en "no hacer nada [posiciaon 0]"
        #si la venta mejor cambió, hay que moificar la venta buena original a 0 (opcional)


    def tagWParms(self, df , Swindow): 

        df1 = pd.DataFrame()

        df1['Open'] = self.data.Open.to_series()
        df1['High'] = self.data.High.to_series()
        df1['Low'] = self.data.Low.to_series()
        df1['Close'] = self.data.Close.to_series()
        #df1 = df.copy(deep=True)

        dataMinor = df1.copy()
        dataMax = df1.copy()    
        dataMinor['mn'] = dataMinor.iloc[argrelextrema(dataMinor.Close.values, np.less_equal, order=Swindow)[0]]['Close']
        dataMinor['mx'] = dataMinor.iloc[argrelextrema(dataMinor.Close.values, np.greater_equal, order=Swindow)[0]]['Close']
        dataMinor['position'] = 0
        dataMinor.loc[dataMinor['mn'].notnull(),'position'] = 1.0 
        dataMinor.loc[dataMinor['mx'].notnull(),'position'] = -1.0 
        
        #dataMax['mn'] = dataMax.iloc[argrelextrema(dataMax.Close.values, np.less_equal, order=Lwindow)[0]]['Close']
        #dataMax['mx'] = dataMax.iloc[argrelextrema(dataMax.Close.values, np.greater_equal, order=Lwindow)[0]]['Close']
        #dataMax['position'] = 0
        #dataMax.loc[dataMax['mn'].notnull(),'position'] = 2.0 
        #dataMax.loc[dataMax['mx'].notnull(),'position'] = -2.0 

        #data = self.merge(dataMinor,dataMax)
        data = dataMinor
        #print(f"VALORES ANTES:\n {dataN['position'].value_counts()}")

        return data
      



    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize all the classes
        global dataN
        print("Window: %d"%(self.n1))
        super().init()       
        dataN = self.tagWParms(self.data, self.n1)                
        dataN = dataN.drop(columns=['mn', 'mx'])
        #print(f"VALORES ANTES:\n {dataN['position'].value_counts()}")
        dataN = self.tagForOptimumSales(dataN, commission = self.commission) #Primera Pasada
        #print(f"VALORES EN MEDIO:\n {dataN['position'].value_counts()}")
        dataN = self.tagForOptimumSales(dataN, commission = self.commission) #Segunda Pasada
        ################################PROPAGATE POSITION##################################
        dataN = self.etiq(dataN, threshold=0.15)        #Propagación de de posición a un bloque
        #print(f"VALORES DESPUES:\n {dataN['position'].value_counts()}")
        #print(f"VALORES DESPUES:\n {dataN['propagated_position'].value_counts()}")

        self.set_signal(pd.Series(dataN['position']),plot=True)
        #data.to_csv('/content/gdrive/My Drive/Colab Notebooks/BITSO/data/datasetEtiquetado.csv')        



In [ ]:
btW1 = Backtest(databt, Agrelextrema1W, cash=400000, commission=.0065, trade_on_close=True) 

In [ ]:
pdW1Btr = btW1.run()

In [ ]:
pdW1Btr

https://www.investopedia.com/terms/d/drawdown.asp

What is a Drawdown?
A drawdown is a peak-to-trough decline during a specific period for an investment, trading account, or fund. A drawdown is usually quoted as the percentage between the peak and the subsequent trough. If a trading account has 10,000 in it, and the funds drop to 9,000 before moving back above 10,000, then the trading account witnessed a 10% drawdown.

In [ ]:
#^ = venta
#v = compra

btW1.plot()

In [ ]:
dataN.loc['2018-11-07 12:00:00']

Para ganar más posiciones al 0:
* Agregar propagación de posiciones para crear bloques. 
* No va a servir con el backtesting
* Agregar clases de ventas y compras óptimas
* Para probar en backtesting tomar la peor posición de cada bloque
* Usando función existente:
  * Crear columna de posicion nueva para comparar. 
  * con la funcion existente propagar 
  * Crear un campo nuevo llamado bloque
  * Hay tantos bloques como posiciones originales hay en la serie
  * Separar el peor de cada bloque para backtesting (de lo contrario BT no funciona)
  * El Delta entre posiciones propagadas debe ser entre la posición contraria (peor del bloque) anteriror. No contra la posicion propagada anterior en el bloque.

In [ ]:
dataN.to_csv('/content/gdrive/My Drive/Colab Notebooks/BINANCE/data/Binance_BTCUSDT_1h_labeled_1_window.csv')

In [ ]:
stats1W, heatmap1W = btW1.optimize(n1 = [1,2,4,6,7,8,9,10,11,12,13,14,15,16,24,45,90,180,360,720,1095,1440,2190,4380,8760],
                  n2 = [0],
                  #constraint=lambda p: p.n2 > p.n1,
                  #maximize='Win Rate [%]',
                  maximize='Win Rate [%]',
                  return_heatmap=True
                  )

In [ ]:
hmW1 = heatmap1W.groupby(['n1']).mean()
hmW1

In [ ]:
stats1W, heatmap1W = btW1.optimize(n1 = [1,2,4,6,7,8,9,10,11,12,13,14,15,16,24,45,90,180,360,720,1095,1440,2190,4380,8760],
                  n2 = [0],
                  #constraint=lambda p: p.n2 > p.n1,
                  #maximize='Win Rate [%]',
                  maximize='Return [%]',
                  return_heatmap=True
                  )

In [ ]:
hmW1 = heatmap1W.groupby(['n1']).mean()
hmW1

In [ ]:
plt.figure(figsize=(15,10))
#sns.heatmap(heatmap1W[:-1], cmap='viridis')
plot_heatmaps(heatmap1W, agg='mean')

Resampling to Daily

In [ ]:
data_daily_resampled = data.resample('D').mean()

In [ ]:
data_daily_resampled.head()

In [ ]:
date_rng = pd.date_range(start='2017-12-24 00:00:00', end='2017-12-25 23:00:00', freq='H')
date_rng

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=dataN.index, y=dataN['Close'],
                    mode='lines',
                    name='Close'))

fig.add_trace(go.Scatter(x=dataN[dataN['position'] == -2].index, y=dataN[dataN['position'] == -2]['Close'],
                    mode='markers',
                    marker = dict(size=12, color= 'blue',symbol= 'star-triangle-up'),
                    name='Top Sale'))
fig.add_trace(go.Scatter(x=dataN[dataN['position'] == -1].index, y=dataN[dataN['position'] == -1]['Close'],
                    mode='markers',
                    marker = dict(size=12,color= 'green',symbol= 'star-triangle-up' ),                    
                    name='Sale'))
fig.add_trace(go.Scatter(x=dataN[dataN['position'] == 1].index, y=dataN[dataN['position'] == 1]['Close'],
                    mode='markers',
                    marker = dict(size=12,color= 'orange',symbol= 'star-triangle-down'),                    
                    name='Buy'))
fig.add_trace(go.Scatter(x=dataN[dataN['position'] == 2].index, y=dataN[dataN['position'] == 2]['Close'],
                    mode='markers',
                    marker = dict(size=12,color= 'red',symbol= 'star-triangle-down' ),                    
                    name='Top Buy'))

fig.show()

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=dataN.index, y=dataN['Close'],
                    mode='lines',
                    name='Close'))

fig.add_trace(go.Scatter(x=dataN[dataN['propagated_position'] == -2].index, y=dataN[dataN['propagated_position'] == -2]['Close'],
                    mode='markers',
                    marker = dict(size=12, color= 'blue',symbol= 'star-triangle-up'),
                    name='Top Sale'))
fig.add_trace(go.Scatter(x=dataN[dataN['propagated_position'] == -1].index, y=dataN[dataN['propagated_position'] == -1]['Close'],
                    mode='markers',
                    marker = dict(size=12,color= 'green',symbol= 'star-triangle-up' ),                    
                    name='Sale'))
fig.add_trace(go.Scatter(x=dataN[dataN['propagated_position'] == 1].index, y=dataN[dataN['propagated_position'] == 1]['Close'],
                    mode='markers',
                    marker = dict(size=12,color= 'orange',symbol= 'star-triangle-down'),                    
                    name='Buy'))
fig.add_trace(go.Scatter(x=dataN[dataN['propagated_position'] == 2].index, y=dataN[dataN['propagated_position'] == 2]['Close'],
                    mode='markers',
                    marker = dict(size=12,color= 'red',symbol= 'star-triangle-down' ),                    
                    name='Top Buy'))

fig.show()

In [ ]:
dataN['propagated_position'].value_counts()

In [ ]:
dataN['position'].value_counts()

In [ ]:
for i in dataN.index:
  if dataN[dataN['position_block'] == i].size > 1:
    print(dataN[dataN['position_block'] == i])
    break

In [ ]:
len(dataN[dataN['position_block'] == '2017-12-24 06:00:00'])

In [ ]:
dataN[dataN['position_block'] == '2017-07-01 20:00:00']

In [ ]:
dataN.head(5)

In [ ]:
dataN.loc[date_rng]

In [ ]:
dataN[dataN['position_block'] == '2017-12-13 18:00:00']

Hacer back testing del Mejor y del Peor de cada bloque
La propagación hacer la sólo hacia adelante

In [ ]:
##################################################################################################################################
###### Implementacion de estrategia con señal generada a partir de argrelextrema y propagación de posiciones en  de Bloques#######
##################################################################################################################################
from backtesting.lib import SignalStrategy, TrailingStrategy

class AgrelextremaWorst(SignalStrategy,TrailingStrategy):
    n1 = 9
    n2= 0
    commission = 0.0065

    d = timedelta(hours=1)

    def previousPosition(self, df, i):
      if i > firstVI :
        prev = i - self.d
      else: prev = i

      while (prev > firstVI and df.loc[prev, 'position'] == 0):
        prev = prev - self.d
      return prev
    
    #Obtiene la mejor y la peor posicion de un bloque
    def getBestWorstPos(self, dfBlock):

      #print(dfBlock)
      wIndex = 0
      bIndex = 0
      pos = 0

      if (dfBlock.propagated_position.max() > 0 ): #Then compra
        best = dfBlock.Close.min()
        worst = dfBlock.Close.max()
        pos = 1
      elif dfBlock.propagated_position.min() < 0: #then venta
        best = dfBlock.Close.max()
        worst = dfBlock.Close.min()
        pos = -1
      else:
        pos = 0
        best = -1
        worst = -1

      wIndex = dfBlock[dfBlock['Close'] == worst].index
      bIndex = dfBlock[dfBlock['Close'] == best].index


      return best, worst, bIndex, wIndex, pos

    def tagWorse(self, df):

      firstVI = df.first_valid_index
      lastVI = df.last_valid_index

      dataWorse = df.copy(deep=True)

      dataWorse['positionW'] = 0

      for i in dataWorse.index:
        dBlock = dataN[dataN['position_block'] == i]
        if len(dBlock) > 1:
          print(i,' ',len(dBlock))
        if dBlock.propagated_position.min() != 0:
          bAmt, wAmt, bIdx, wIdx, pstn = self.getBestWorstPos(dBlock)        
          dataWorse.loc[wIdx,'positionW'] = pstn

      return dataWorse




    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize all the classes
        global dataN
        super().init()       
        ################################Etiqueta peor##################################
        dataN = self.tagWorse(dataN)        
        #print(f"VALORES DESPUES:\n {dataN['position'].value_counts()}")
        #print(f"VALORES DESPUES:\n {dataN['propagated_position'].value_counts()}")

        self.set_signal(pd.Series(dataN['positionW']), plot=True)
        #data.to_csv('/content/gdrive/My Drive/Colab Notebooks/BITSO/data/datasetEtiquetado.csv')        

In [ ]:
btWrst = Backtest(databt, AgrelextremaWorst, cash=400000, commission=0.0065, trade_on_close=True) 

In [ ]:
pdWrst = btWrst.run()

In [ ]:
pdWrst

In [ ]:
dataN

In [ ]:
dataN.to_csv('/content/gdrive/My Drive/Colab Notebooks/BINANCE/data/Binance_BTCUSDT_1h_labeled_1_window_propagated_blocks.csv')

In [ ]:
#^ = venta
#v = compra

btWrst.plot()

##Buscar Posiciones consecutivas redundantes
como 2017-08-24 10:00:00 y 2017-08-24 19:00:00 ambas ventas consecutivas

Las pérdidas son atribuibles a posiciones redundantes. Principalmente de venta. Revisar código en función

In [ ]:
dataTshoot = dataN[dataN['position'] != 0].copy(deep=True)

In [ ]:
len(dataTshoot)

In [ ]:
dataTshoot['consecutivePos'] = np.sign(dataTshoot.position.diff().fillna(0)).shift(-1).fillna(0)

In [ ]:
dataTshoot['consecutivePos'].value_counts()

In [ ]:
date_rng = pd.date_range(start='2017-08-24 00:00:00', end='2017-08-25 23:00:00', freq='H')
date_rng

In [ ]:
dataTshoot[dataTshoot['consecutivePos'] == 0]

In [ ]:
dataN.loc['2017-08-24 12:00:00']

In [ ]:
dataN.loc['2017-08-24 19:00:00']

In [ ]:
dataN['propagated_position'].value_counts()

In [ ]:
dataN['positionW'].value_counts()

In [ ]:
dictBlock = {}
for i in dataN.index:
  dBlock = dataN[dataN['position_block'] == i]
  if i not in dictBlock:
    dictBlock[i] = len(dBlock)

In [ ]:
len(dictBlock)

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=dataN.index, y=dataN['Close'],
                    mode='lines',
                    name='Close'))

fig.add_trace(go.Scatter(x=dataN[dataN['positionW'] == -2].index, y=dataN[dataN['positionW'] == -2]['Close'],
                    mode='markers',
                    name='Top Sale'))
fig.add_trace(go.Scatter(x=dataN[dataN['positionW'] == -1].index, y=dataN[dataN['positionW'] == -1]['Close'],
                    mode='markers',
                    name='Sale'))
fig.add_trace(go.Scatter(x=dataN[dataN['positionW'] == 1].index, y=dataN[dataN['positionW'] == 1]['Close'],
                    mode='markers',
                    name='Buy'))
fig.add_trace(go.Scatter(x=dataN[dataN['positionW'] == 2].index, y=dataN[dataN['positionW'] == 2]['Close'],
                    mode='markers',
                    name='Top Buy'))

fig.show()

In [ ]:
"""
for i in dictBlock.keys():
  print(i , " ",dictBlock[i])
"""

Resample data to 4 hours next

In [ ]:
result = seasonal_decompose(data['Close'][date_rng], model='multiplicative', freq=1)

In [ ]:
date_rng4H = pd.date_range(start='2017-12-15 00:00:00', end='2017-12-23 12:00:00', freq='4H')
#date_rng4H

In [ ]:
# Resampling data to 4 
data_4H_resampled = data.resample('4H').mean()

In [ ]:
data_4H_resampled['Close'][date_rng4H].head()

In [ ]:
result4H = seasonal_decompose(data_4H_resampled['Close'][date_rng4H], model='multiplicative', freq=4)

result4H.plot()
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
autocorrelation_plot(data_4H_resampled['Close'][date_rng4H])

In [ ]:
date_rng6H = pd.date_range(start='2017-12-15 00:00:00', end='2017-12-23 12:00:00', freq='6H')
date_rng6H

In [ ]:
data_6H_resampled = data.resample('6H').mean()
data_6H_resampled.head()

In [ ]:

result6H = seasonal_decompose(data_6H_resampled['Close'][date_rng6H], model='multiplicative', freq=1)
result6H.plot()
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
autocorrelation_plot(data_6H_resampled['Close'][date_rng6H])

In [ ]:
data_12H_resampled = data.resample('12H').mean()
data_12H_resampled.head()

In [ ]:
date_rng12H = pd.date_range(start='2017-12-15 00:00:00', end='2017-12-23 12:00:00', freq='12H')
date_rng12H

In [ ]:

result12H = seasonal_decompose(data_12H_resampled['Close'][date_rng12H], model='multiplicative', freq=1)
result12H.plot()
plt.show()

In [ ]:
data_12H_resampled.head()

In [ ]:
plt.figure(figsize=(20,15))
#autocorrelation_plot(data_12H_resampled['Close'][date_rng12H])
autocorrelation_plot(data_12H_resampled['Close'])

In [ ]:
plt.figure(figsize=(20,15))
#autocorrelation_plot(data['Close'][date_rng])
autocorrelation_plot(data['Close'])

Encontrar relación entre valor actual con valor dentro de ventana de tiempo.

https://robjhyndman.com/hyndsight/tscv/


bitcoin datasources:

* http://www.coindesk.com/price
* http://www.investing.com/currencies/btc-usd-historical-data
* http://www.quandl.com/c/markets/bitcoin-data
* https://bitcoinchain.com/markets



In [ ]:
######################################################################
####### For this to work I need to set-up the function with Agrelextrema signal generation in the strategy class
#######
#%%time

#stats, heatmap = bt.optimize(d_rsi=range(10, 35, 5),
#                  w_rsi=range(10, 35, 5),
#                  level=range(30, 80, 10),
#                  maximize='Return [%]',
#                  return_heatmap=True
#                  )